# **ModelHub - Whole Body CT Segmentation**

This notebook provides a minimal working example of TotalSegmentator, a tool for the segmentation of 104 anatomical structures from CT images. The model was trained using a wide range of imaging CT data of different pathologies from several scanners, protocols and institutions.

We test TotalSegmentator by implementing an end-to-end (cloud-based) pipeline on publicly available whole body CT scans hosted on the [Imaging Data Commons (IDC)](https://portal.imaging.datacommons.cancer.gov/), starting from raw DICOM CT data and ending with a DICOM SEG object storing the segmentation masks generated by the AI model. The testing dataset we use is external and independent from the data used in the development phase of the model (training and validation) and is composed by a wide variety of image types (from the area covered by the scan, to the presence of contrast and various types of artefacts).

The way all the operations are executed - from pulling data, to data postprocessing, and the standardisation of the results - have the goal of promoting transparency and reproducibility. Furthermore, this notebook is part of [a collection of code, notebooks and Docker containers](https://github.com/AIM-Harvard/mhub/blob/main/mhub/totalsegmentator/notebooks/totalsegmentator_mwe.ipynb) we are developing with the goal of making a wide range of machine learning models for medicine available through a standardized I/O framework.



Please cite the following article if you use this code or pre-trained models:

Wasserthal, J., Meyer, M., Breit, H.C., Cyriac, J., Yang, S. and Segeroth, M., 2022. TotalSegmentator: robust segmentation of 104 anatomical structures in CT images. arXiv preprint arXiv:2208.05868, [
https://doi.org/10.48550/arXiv.2208.05868]( 	
https://doi.org/10.48550/arXiv.2208.05868).

The original code is published on
[GitHub](https://github.com/wasserth/TotalSegmentator)  using the [Apache-2.0 license](https://github.com/wasserth/TotalSegmentator/blob/master/LICENSE).

### **Disclaimer**

The code and data of this repository are provided to promote reproducible research. They are not intended for clinical care or commercial use.

The software is provided "as is", without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.

# **Environment Setup**

This demo notebook is intended to be run using a GPU.

To access a free GPU on Colab:
`Edit > Notebooks Settings`.

From the dropdown menu under `Hardware accelerator`, select `GPU`. Let's check the Colab instance is indeed equipped with a GPU.

This notebook when intended to use in a workflow in Terra or CGC-Seven bridges, can also be run with out gpu

In [ ]:
# #install s5cmd
# !wget "https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz"
# !tar -xvzf "s5cmd_2.0.0_Linux-64bit.tar.gz"
# !rm "s5cmd_2.0.0_Linux-64bit.tar.gz"
# !mv s5cmd /usr/local/bin/s5cmd

In [ ]:
import os
import sys
import yaml
import time
import random
from pathlib import Path

# useful information
curr_dir   = Path().absolute()
curr_droid = !hostname
curr_pilot = !whoami

print(time.asctime(time.localtime()))
print("\nCurrent directory :{}".format( curr_dir))
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])
print("Python version    :", sys.version.split('\n')[0])


# ----------------------------------------

import numpy as np
import pandas as pd

print("Numpy version                : ", np.__version__)

The authentication to Google is necessary to run BigQuery queries.

Every operation throughout the whole notebook (BigQuery, fetching data from the IDC buckets) is completely free. The only thing that is needed in order to run the notebook is the set-up of a Google Cloud project. In order for the notebook to work as intended, you will need to specify the name of the project in the cell after the authentication one.

In [ ]:
# # when running on Terra or Seven bridges this cell can be commented out as 
# # we will not need any authentication to google cloud except when we want to use 
# # bigquery to download dicom files. In that case, authentication is taken care of 
# # by using a service account

# from google.colab import auth
# auth.authenticate_user()

In [ ]:
## from google.colab import files 

## from google.cloud import storage
## from google.cloud import bigquery as bq

## INSERT THE ID OF YOUR PROJECT HERE!
## project_id = "idc-external-030" 

In [ ]:
# %%capture
# # utility to make yaml files easily editable in a notebook cell
# !pip install yamlmagic 

In [ ]:
%load_ext yamlmagic

Throughout this Colab notebook, for image pre-processing we will use [Plastimatch](https://plastimatch.org), a reliable and open source software for image computation. We will be running Plastimatch using the simple [PyPlastimatch](https://github.com/AIM-Harvard/pyplastimatch/tree/main/pyplastimatch) python wrapper. 

In [ ]:
# %%capture
# !apt install plastimatch 

In [ ]:
#check plastimatch was correctly installed
!plastimatch --version 

Install Apache's subversion. 

We will use subversion to clone only specific subfolders of the mhub repository.

In [ ]:
# %%capture
# !apt install subversion 

---

Start by cloning the AIMI hub repository on the Colab instance.

The AIMI hub repository stores all the code we will use for pulling, preprocessing, processing, and postprocessing the data for this use case - as long as the others shared through AIMI hub.

In [ ]:
%%capture
# !svn checkout https://github.com/MHubAI/mhubio/trunk/mhubio mhubio
# !svn checkout https://github.com/MHubAI/mhubio/trunk/mhubio/utils/ymldicomseg mhubai/ymldicomseg
# !svn checkout https://github.com/MHubAI/models/trunk/models/totalsegmentator mhubai/totalsegmentator

!svn checkout https://github.com/vkt1414/mhubio/trunk/mhubio mhubio
!svn checkout https://github.com/vkt1414/mhubio/trunk/mhubio/utils/ymldicomseg mhubai/ymldicomseg
!svn checkout https://github.com/vkt1414/models/trunk/models/totalsegmentator mhubai/totalsegmentator

To organise the DICOM data in a more common (and human-understandable) fashion after downloading those from the buckets, we will make use of [DICOMSort](https://github.com/pieper/dicomsort). 

DICOMSort is an open source tool for custom sorting and renaming of dicom files based on their specific DICOM tags. In our case, we will exploit DICOMSort to organise the DICOM data by `PatientID` and `Modality` - so that the final directory will look like the following:

```
data/raw/nsclc-radiomics/dicom/$PatientID
 └─── CT
       ├─── $SOPInstanceUID_slice0.dcm
       ├─── $SOPInstanceUID_slice1.dcm
       ├───  ...
       │
      RTSTRUCT 
       ├─── $SOPInstanceUID_RTSTRUCT.dcm
      SEG
       └─── $SOPInstanceUID_RTSEG.dcm

```

We will also use DCMQI for converting the resulting segmentation into standard DICOM SEG objects.

In [ ]:
# %%capture
# dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.2.5/dcmqi-1.2.5-linux.tar.gz"
# dcmqi_download_path = "/content/dcmqi-1.2.5-linux.tar.gz"
# dcmqi_path = "/content/dcmqi-1.2.5-linux"

# !wget -O $dcmqi_download_path $dcmqi_release_url

# !tar -xvf $dcmqi_download_path

# !mv $dcmqi_path/bin/* /bin

---

Let's now install example-specific python dependencies we will need.

In [ ]:
# %%capture

# !pip install thedicomsort
# !pip install pyplastimatch nnunet
# !pip install TotalSegmentator

Provided everything was set up correctly, we can run the BigQuery query and get all the information we need to download the testing data from the IDC platform.

For this specific use case, we are going to be working with the "CT lymph nodes" collection hosted on IDC - which groups a collections of series that are close to whole body CT scans.

In [ ]:
#JSONServiceAccountFile='graceful-goods-375814-d6d8e1553699.json'
SeriesInstanceUIDs=['1.2.840.113654.2.55.269907831379668070404580302097056025638','1.3.6.1.4.1.14519.5.2.1.7009.9004.231985094753394017986102999426']
FastModeStatus= True


In [ ]:
# ##authenticating google cloud with a service account to run bigquery
# #when the new IDC storage schema is live, there will be no need to use service account

# from google.cloud import storage
# from google.cloud import bigquery as bq
# from google.oauth2 import service_account
# !gsutil cp gs://fc-859bd570-893c-4e71-a677-875d7d0d7af3/graceful-goods-375814-d6d8e1553699.json .
# credentials = service_account.Credentials.from_service_account_file(
#      os.path.join(curr_dir,'graceful-goods-375814-d6d8e1553699.json'), scopes=["https://www.googleapis.com/auth/cloud-platform"],)
# bq_client = bq.Client(credentials=credentials, project=credentials.project_id)


In [ ]:
# selection_query = """
# select distinct
# nlst.SeriesInstanceUID,
# idc.gcs_url,
# CONCAT("cp ",REPLACE(idc.gcs_url, "gs://", "s3://"), " data/idc_data/") as s5cmd_cp_url

#  from `graceful-goods-375814.terra.nlst` nlst

#  join `bigquery-public-data.idc_current.dicom_all` idc

#  on nlst.SeriesInstanceUID = idc.SeriesInstanceUID

# """

# cohort_df = bq_client.query(selection_query).to_dataframe()
# #cohort_df=cohort_df[cohort_df["SeriesInstanceUID"]==SeriesInstanceUID]

In [ ]:
#get the path of the zip file containing csv file in the current working directory
import glob
#zip_file_path='/home/vamsi/Downloads/result.zip'

zip_file_path = glob.glob('*.zip')[0]
#zip_file_path='/content/drive/MyDrive/result.zip'
!unzip $zip_file_path

csv_file_path = glob.glob('*.csv')[0]
#csv_file_path= '/home/vamsi/Downloads/result.csv'

In [ ]:
# %%capture
# !pip install modin[ray]

In [ ]:
#import ray
#!pip install dask
#!pip install pandas==1.5.3
import os
#os.environ["MODIN_ENGINE"] = "ray" 
#import modin.pandas as pd

import pandas as pd
#import dask.dataframe as dd
#import csv
#import pandas as pd
#cohort_df = dd.read_csv(csv_file_path, delimiter=',',quoting=csv.QUOTE_NONE, encoding='utf-8').compute()
start_time = time.time()
cohort_df=pd.read_csv(csv_file_path, delimiter=',', encoding='utf-8')
read_time=time.time() -start_time
print('read in '+str(read_time)+ '  seconds')
#cohort_df=pd.read_csv(csv_file_path, delimiter=',', encoding='utf-8')

In [ ]:
# # N.B. - this works as intended only if the BQ query parses data from a single dataset
# # if not, feel free to set the name manually!
# dataset_name = cohort_df["collection_id"].values[0]
# dataset_name

In [ ]:
# create the directory tree
!rm -r data
!mkdir -p data
!mkdir -p data/idc_data
!mkdir -p data/input_data 
!mkdir -p data/output_data 
!mkdir -p data/sorted

# **Parsing Cohort Information from BigQuery Tables**

We can check the various fields of the table we populated by running the BigQuery query.

This table will store one entry for each DICOM file in the dataset (therefore, expect thousands of rows!)

In [ ]:
# pat_id_list = sorted(list(set(cohort_df["PatientID"].values)))

# print("Total number of unique Patient IDs:", len(pat_id_list))

# display(cohort_df.info())
# display(cohort_df.head())

# **Setup mhubio**

`mhbio` is the module of MHub that deals with all of the basic operations shared between a large majority of the models.

Let's import all of the modules we will need to run the `TotalSegmentator` pipeline from end to end.


In [ ]:
# !pip install pyplastimatch

In [ ]:
sys.path.append('.')

from mhubio.core import Config, DataType, FileType, CT, SEG

from mhubio.modules.importer.UnsortedDicomImporter import UnsortedInstanceImporter
from mhubio.modules.importer.DataSorter import DataSorter
from mhubio.modules.convert.NiftiConverter import NiftiConverter
from mhubio.modules.convert.DsegConverter import DsegConverter
from mhubio.modules.organizer.DataOrganizer import DataOrganizer

from mhubai.totalsegmentator.utils.TotalSegmentatorRunner import TotalSegmentatorRunner

For instance, the workflow for this example looks like the following:

```
DICOM CT >> NIfTI >> TotalSegmentator >> NIfTI >> DICOM SEG 
```

We want to start from DICOM CT data and save the results in DICOM SEG format. We are therefore going to need the relevant `DataType`(s) and `FileType`(s) imported (`CT` and the `SEG`) from our `Config` module. 

We will also need to import all the converters to run the aforementioned operations (`UnsortedInstanceImporter`, `DataSorter`, `NiftiConverter`, `DsegConverter`, `DataOrganizer`) and the model runner (`TotalSegmentatorRunner`).

For more in-depth explanation of the modules, see the following sections.


# **Running the Analysis for a Single Patient**

The following cells run all the processing pipeline, from pre-processing to post-processing.

Let's start by slicing the datafram storing the metadata we pulled from IDC to get a single series from a single patient.

In [ ]:
# # randomly select one patient from the cohort
# pat_id = random.choice(cohort_df["PatientID"].values)
# patient_df = cohort_df[cohort_df["PatientID"] == pat_id].reset_index(drop = True)

# # select only data for which the modality is CT 
# #patient_df = patient_df[patient_df["Modality"] == "CT"].reset_index(drop = True)

# # if more than one series are available for the selected patient, pick one
# if len(np.unique(patient_df["SeriesInstanceUID"].values)) > 1:
#   series_uid = random.choice(patient_df["SeriesInstanceUID"].values)
#   patient_df = patient_df[patient_df["SeriesInstanceUID"] == series_uid].reset_index(drop = True)

# # sanity check
# assert len(np.unique(patient_df["SeriesInstanceUID"].values)) == 1

# display(patient_df.info())
# display(patient_df.head())

Write a custom configuration file containing the specifics for all the MHub modules we're going to use, using the `%%writefile` magik (from the `yamlmagic` package).

This file is going to be tailored to this specific use case and example.

The config file stores all the information needed by the different modules, such as the path to a given file or folder, to execute the end-to-end pipeline.

For instance, we specify `data_base_dir` as a `general` argument all the modules are going to share. Then, we specify the module-specific arguments.

For instance, we specify the directory where the DICOM data to be sorted are for the `UnsortedInstanceImporter`, by adding `input_dir` to the config file. We can also specify the structure of the sorted directory by setting the `structure` argument for the `DataSorter`, or which `TotalSegmentator` model to run.

In [ ]:
%%writefile totalsegmentator_config.yml

general:
  data_base_dir: /content/data
modules:
  UnsortedInstanceImporter:
    input_dir: /content/data/idc_data
  DataSorter:
    base_dir: /content/data/sorted
    structure: '%SeriesInstanceUID/dicom/%SOPInstanceUID.dcm'
  DsegConverter:
    dicomseg_json_path: mhubai/totalsegmentator/config/dicomseg_metadata_whole.json
    #dicomseg_yml_path: mhubai/totalsegmentator/config/dseg.yml
    skip_empty_slices: True
  TotalSegmentatorRunner:
    use_fast_mode: FastModeStatus
    use_multi_label_output: True

In [ ]:
with open('totalsegmentator_config.yml', 'r') as file:
    config = yaml.safe_load(file)

# Update the value of use_fast_mode with the value of FastModeStatus
config['general']['data_base_dir'] = os.path.join(curr_dir,'data')
config['modules']['UnsortedInstanceImporter']['input_dir'] = os.path.join(curr_dir,'data/idc_data')
config['modules']['DataSorter']['base_dir'] = os.path.join(curr_dir,'data/sorted')
config['modules']['TotalSegmentatorRunner']['use_fast_mode'] = FastModeStatus

# Write the updated dictionary back to the YAML file
with open('totalsegmentator_config.yml', 'w') as file:
    yaml.safe_dump(config, file)

After defining a config file for our use case, we can initialize a `Config` object using such `yml` file.

The `Config` object is passed along to all the modules, and keeps track of all of the information that need to shared among these modules.

In [ ]:
# config
config = Config('totalsegmentator_config.yml')
config.verbose = True  

In the next cells, we define a utility function to pull data from the Imaging Data Commons - starting from the DataFrame `patient_df` we have previously defined - and then cross-load the data from the IDC Buckets to this Colab instance.

In [ ]:
# def download_patient_data(download_path, patient_df):

#   """
#   Download raw DICOM data and run dicomsort to standardise the input format.
#   Arguments:
#     download_path : required - path to the folder where the raw data will be downloaded.
#     patient_df    : required - Pandas dataframe storing all the information required
#                                to pull data  from the IDC buckets.
#   """

#   gs_file_path = "gcs_paths.txt"
#   patient_df["gcs_url"].to_csv(gs_file_path, header = False, index = False)

#   pat_id = patient_df["PatientID"].values[0]
#   download_path = os.path.join(download_path, pat_id)

#   if not os.path.exists(download_path): os.mkdir(download_path)

#   start_time = time.time()
#   print("Copying files from IDC buckets to %s..."%(download_path))

#   !cat $gs_file_path | gsutil -q -m cp -Ir $download_path >> /dev/null

#   elapsed = time.time() - start_time
#   print("Done in %g seconds."%elapsed)

In [ ]:
# def download_patient_data(download_path,series_id):

#   """
#   Download raw DICOM data and run dicomsort to standardise the input format.
#   Arguments:
#     download_path : required - path to the folder where the raw data will be downloaded.
#     patient_df    : required - Pandas dataframe storing all the information required
#                                to pull data  from the IDC buckets.
#   """

#   gs_file_path = "s5cmd_manifest.txt"
#   cohort_df = bq_client.query(selection_query).to_dataframe()
#   cohort_df=cohort_df[cohort_df['SeriesInstanceUID']==series_id]
#   cohort_df["s5cmd_cp_url"].to_csv(gs_file_path, header = False, index = False)

#   download_path = os.path.join(download_path)

#   if not os.path.exists(download_path): os.mkdir(download_path)

#   start_time = time.time()
#   print("Copying files from IDC buckets to %s..."%(download_path))

#   #!cat $gs_file_path | gsutil -q -m cp -Ir $download_path >> /dev/null
#   !s5cmd --no-sign-request --endpoint-url https://storage.googleapis.com run s5cmd_manifest.txt

#   elapsed = time.time() - start_time
#   print("Done in %g seconds."%elapsed)

In [ ]:
def download_patient_data(download_path,series_id):

  """
  Download raw DICOM data and run dicomsort to standardise the input format.
  Arguments:
    download_path : required - path to the folder where the raw data will be downloaded.
    patient_df    : required - Pandas dataframe storing all the information required
                               to pull data  from the IDC buckets.
  """
  global cohort_df
  gs_file_path = "s5cmd_manifest.txt"
  #cohort_df = bq_client.query(selection_query).to_dataframe()
  series_df=cohort_df[cohort_df['SeriesInstanceUID']==series_id]
  series_df["s5cmd_cp_url"].to_csv(gs_file_path, header = False, index = False)
  #remove double quotes from the manifest file
  !sed -i 's/"//g' s5cmd_manifest.txt  

  download_path = os.path.join(download_path)

  if not os.path.exists(download_path): os.mkdir(download_path)

  start_time = time.time()
  print("Copying files from IDC buckets to %s..."%(download_path))

  #!cat $gs_file_path | gsutil -q -m cp -Ir $download_path >> /dev/null
  !s5cmd --no-sign-request --endpoint-url https://storage.googleapis.com run s5cmd_manifest.txt  >> /dev/null

  elapsed = time.time() - start_time
  print("Done in %g seconds."%elapsed)

In [ ]:
import os
import zipfile
from concurrent.futures import ThreadPoolExecutor
from time import sleep
from datetime import datetime
import psutil
import pandas as pd
import matplotlib.pyplot as plt

class MemoryMonitor:
    def __init__(self):
        self.keep_measuring = True

    def measure_usage(self):
        cpu_usage = []
        ram_usage_mb=[]
        time_stamps = []
        start_time = time.time()
        while self.keep_measuring:
            cpu = psutil.cpu_percent()
            ram = psutil.virtual_memory()
            ram_total_mb = psutil.virtual_memory().total / 1024 / 1024
            ram_mb = (ram.total - ram.available) / 1024 / 1024
            cpu_usage.append(cpu)
            ram_usage_mb.append(ram_mb)
            time_stamps.append(time.time()- start_time)
            sleep(1)

        return cpu_usage, ram_usage_mb, time_stamps, ram_total_mb

In [ ]:
# runTimeStatistics=pd.DataFrame()

# def download_and_process_series(series_id):
#     global runTimeStatistics
#     log=pd.DataFrame()
#     log['SeriesInstanceUID']=[series_id]
#     start_time = time.time()
#     download_patient_data("/content/data/idc_data", series_id)
#     download_time=time.time()- start_time
#     log[series_id]['download_time']=download_time

#     start_time = time.time()
#     importer = UnsortedInstanceImporter(config)
#     importer.execute()
#     importer_time=time.time()- start_time
#     log[series_id]['importer_time']=importer_time

#     start_time = time.time()
#     DataSorter(config).execute()
#     DataSorter_time=time.time()- start_time

#     start_time = time.time()
#     NiftiConverter(config).execute()
#     NiftiConverter_time=time.time()- start_time

#     start_time = time.time()
#     TotalSegmentatorRunner(config).execute()
#     TotalSegmentatorRunner_time=time.time()- start_time

#     start_time = time.time()
#     DsegConverter(config).execute()
#     DsegConverter_time=time.time()- start_time

#     start_time = time.time()
#     archive_fn = "{}.zip".format(series_id)
#     archiving_time=time.time()- start_time
#     # Annotate on the plots
#     try:
#         os.remove(archive_fn)
#     except OSError:
#         pass
#     with zipfile.ZipFile(archive_fn, "w", zipfile.ZIP_DEFLATED) as zip_file:
#         zip_file.write("data/sorted/{}/seg.dcm".format(series_id))

#     # Clean up data directories
#     os.system("rm -r data/ s5cmd_manifest.txt")
#     os.system("mkdir -p data/")
#     os.system("mkdir -p data/idc_data")
#     os.system("mkdir -p data/input_data")
#     os.system("mkdir -p data/output_data")

In [ ]:
def download_and_process_series(series_id):
    log = pd.DataFrame({'SeriesInstanceUID': [series_id]})
    start_time = time.time()
    download_patient_data(os.path.join(curr_dir,'data/idc_data'), series_id)
    download_time = time.time() - start_time
    log['download_time'] = download_time

    start_time = time.time()
    importer = UnsortedInstanceImporter(config)
    importer.execute()
    importer_time = time.time() - start_time
    log['importer_time'] = importer_time

    start_time = time.time()
    DataSorter(config).execute()
    DataSorter_time = time.time() - start_time

    start_time = time.time()
    NiftiConverter(config).execute()
    NiftiConverter_time = time.time() - start_time

    start_time = time.time()
    TotalSegmentatorRunner(config).execute()
    TotalSegmentatorRunner_time = time.time() - start_time

    start_time = time.time()
    DsegConverter(config).execute()
    DsegConverter_time = time.time() - start_time

    start_time = time.time()
    archive_fn = "{}.zip".format(series_id)
    archiving_time = time.time() - start_time
    try:
     os.remove(archive_fn)
    except OSError:
     pass
    !zip -j -r "$archive_fn" "data/sorted/$series_id/seg.dcm" 

    os.system("rm -r data/ s5cmd_manifest.txt")
    os.system("mkdir -p data/")
    os.system("mkdir -p data/idc_data")
    os.system("mkdir -p data/input_data")
    os.system("mkdir -p data/output_data")
    os.system("mkdir -p data/sorted")

    log['DataSorter_time'] = DataSorter_time
    log['NiftiConverter_time'] = NiftiConverter_time
    log['TotalSegmentatorRunner_time'] = TotalSegmentatorRunner_time
    log['DsegConverter_time'] = DsegConverter_time
    log['archiving_time'] = archiving_time
    global runtime_stats
    runtime_stats = runtime_stats.append(log, ignore_index=True)



In [ ]:
runtime_stats = pd.DataFrame(columns=['SeriesInstanceUID','download_time','importer_time','DataSorter_time',
                                      'NiftiConverter_time', 'TotalSegmentatorRunner_time',
                                      'DsegConverter_time',	'archiving_time', 'cpu_usage','ram_usage_mb', 'ram_total_mb'
                                      ])
if __name__ == "__main__":
    for series_id in SeriesInstanceUIDs:
        with ThreadPoolExecutor() as executor:
            monitor = MemoryMonitor()
            mem_thread = executor.submit(monitor.measure_usage)
            try:
                proc_thread = executor.submit(download_and_process_series, series_id)
                proc_thread.result()
            finally:
                monitor.keep_measuring = False
                cpu_usage, ram_usage_mb, time_stamps, ram_total_mb = mem_thread.result()
                
                cpu_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[cpu_idx, runtime_stats.columns.get_loc('cpu_usage')] = [[cpu_usage]]

                ram_usage_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[ram_usage_mb_idx, runtime_stats.columns.get_loc('ram_usage_mb')] = [[ram_usage_mb]]
                
                ram_total_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[ram_total_mb_idx, runtime_stats.columns.get_loc('ram_total_mb')] = [[ram_total_mb]]
                
                
                #runtime_stats.at[runtime_stats['SeriesInstanceUID']==series_id, 'cpu_usage']=[[cpu_usage]]
                #runtime_stats.at[runtime_stats['SeriesInstanceUID']==series_id, 'ram_usage_mb']=[[ram_usage_mb]]
                #runtime_stats.at[runtime_stats['SeriesInstanceUID']==series_id, 'ram_total_mb']=[[ram_total_mb]]

                plt.plot(time_stamps, cpu_usage)
                plt.ylim(0, 100)
                plt.xlabel('Time (s)')
                plt.ylabel('CPU usage (%)')
                plt.show()

                plt.plot(time_stamps, ram_usage_mb)
                plt.ylim(0, ram_total_mb)
                plt.xlabel('Time (s)')
                plt.ylabel('Memory usage (MB)')
                plt.show()

runtime_stats.to_csv('runtime.csv')
import subprocess
subprocess.run(["zip", "-j", "-r", "runtime_stats.zip", os.path.join(curr_dir, "runtime.csv")])





In [ ]:
runtime_stats

In [ ]:
# for series_id in SeriesInstanceUIDs:  
#   download_patient_data("data/idc_data",series_id)

#   importer = UnsortedInstanceImporter(config).execute()
#   DataSorter(config).execute()
#   NiftiConverter(config).execute()
#   TotalSegmentatorRunner(config).execute()
#   DsegConverter(config).execute()
#   # organizer = DataOrganizer(config, set_file_permissions = sys.platform.startswith('linux'))
#   # organizer.setTarget(DataType(FileType.NIFTI, CT), "{}/data/output_data/[i:SeriesID]/[path]".format(curr_dir))
#   # organizer.setTarget(DataType(FileType.DICOMSEG, SEG), "{}/data/output_data/[i:SeriesID]/TotalSegmentator.seg.dcm".format(curr_dir))
#   # organizer.execute()
#   archive_fn = "{}.zip".format(series_id)

#   # import pydicom
#   # seg_series = pydicom.dcm_read(f"output_data/{series_id}/TotalSegmentator.dcm")
#   # series_uid = seg_series.SeriesInstanceUID

#   # # cp TotalSegmentator.seg.dcm to {series_uid}.dcm and zip

#   try:
#      os.remove(archive_fn)
#   except OSError:
#      pass
#   !zip -j -r "$archive_fn" "data/sorted/$series_id/seg.dcm" 
#   !rm -r data/ s5cmd_manifest.txt 
#   !mkdir -p data/
#   !mkdir -p data/idc_data
#   !mkdir -p data/input_data 
#   !mkdir -p data/output_data 
#   !mkdir -p data/sorted

We then import the DICOM data found at `data/idc_data`, as specified in the config file at:

```
general:
  data_base_dir: /content/data
modules:
  UnsortedInstanceImporter:
    input_dir: idc_data
```

In [ ]:
# # import a collection of unsorted DICOM data
# importer = UnsortedInstanceImporter(config)
# importer.execute()

After importing the data, we sort the DICOM files in the fashion specified in the config file - and after, we convert to NIfTI, which is the format accepted by TotalSegmentator.

In [ ]:
# # sort such collection of DICOM data using dicomsort
# DataSorter(config).execute()

In [ ]:
# # convert the DICOM data to NIfTI, as required by TotalSegmentator, using plastimatch
# NiftiConverter(config).execute()

Finally, we run TotalSegmentator using the parameters specified at the `TotalSegmentatorRunner` module of the config file, and convert the results back to DICOM SEG using the parameters set in the `DsegConverter` (`dicomseg_yml_path` and `dicomseg_json_path`).

In [ ]:
# # run the inference phase 
# TotalSegmentatorRunner(config).execute()

In [ ]:
# # convert the results to DICOM SEG
# DsegConverter(config).execute()

As a last step, we run MHub's `DataOrganizer` to organize the output data in a predetermined and standardized structure.

In [ ]:
# # organize data into output folder
# # FIXME: don't save stuff under /app
# organizer = DataOrganizer(config, set_file_permissions = sys.platform.startswith('linux'))
# organizer.setTarget(DataType(FileType.NIFTI, CT), "{}/data/output_data/[i:SeriesID]/[path]".format(curr_dir))
# organizer.setTarget(DataType(FileType.DICOMSEG, SEG), "{}/data/output_data/[i:SeriesID]/TotalSegmentator.seg.dcm".format(curr_dir))
# organizer.execute()

---

# **Downloading the Results Locally**

After the end-to-end processing is done, we can download the results (and the pre-processed input data) running the following cells.

In [ ]:
# %%capture

# archive_fn = "%s.zip"%(SeriesInstanceUID)

# try:
#   os.remove(archive_fn)
# except OSError:
#   pass

# !zip -j -r $archive_fn "{}/data/output_data".format(curr_dir) "{}/data/input_data".format(curr_dir)

In [ ]:
# filesize = os.stat(archive_fn).st_size/1024e03
# print('Starting the download of "%s" (%2.1f MB)...\n'%(archive_fn, filesize))

# files.download(archive_fn)